In [22]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text \
#   langchain-cohere \
#.  cohere    

In [23]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore


In [24]:
#  import nltk
#  nltk.download('punkt')

In [25]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')


# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

In [26]:
directory = '/Users/user/Downloads/colab/data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

docs = load_docs(directory)

In [1]:
#docs

In [28]:
print(len(docs))

19


In [29]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

support@vitafyhealth.com | 1-866-4-Vitafy

Billings Module

October, 2023

All the notifications sen
{'source': '/Users/user/Downloads/colab/data/Clients - Comms Module .docx'}


In [30]:

os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x326fefdd0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x3348cff20>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [31]:
import getpass
import os
import time
from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

In [32]:
chunk_size = 2000  
chunk_overlap = 500  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

In [33]:
index_name = "test-2"
#vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
retriever = vectorstore.as_retriever()

## List of the IDs of all records


In [34]:
# from pinecone.grpc import PineconeGRPC as Pinecone

# index = pc.Index("test-2")
# ids = index.list(namespace='')

# for ids in index.list(namespace=''):
#     print(ids)
    
# #['doc1#v1#chunk1', 'doc1#v1#chunk2', 'doc1#v1#chunk3']

In [2]:
# import os


# from PyPDF2 import PdfReader  # For reading PDF files
# import docx 
# index_name = "test-2"
# directory = 'data'
# # 1. Upload data from 'data' directory
# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type
        
# # 1. Upload data from 'data' directory
# for filename in os.listdir(directory):
#     file_path = os.path.join(directory, filename)
    
#     # Read the content of the document based on its type
#     document_text = read_document(file_path)
    
#     if document_text:  # Only proceed if the document was read successfully
#         # 2. Generate embedding for the document
#         vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

        
#         # 3. Generate a unique vector ID (using the filename without extension)
#         #vector_id = os.path.splitext(filename)[0]  # Unique ID based on filename
        
#         vector_id = str(uuid.uuid4())  # Generate a unique UUID

        
#         # Prepare metadata (optional)
#         metadata = {
#             "source": filename,
#             "page_content": document_text[:100]  # Store first 100 characters as a preview (optional)
#         }
        
#         # Upsert the vector into Pinecone
#         index.upsert(vectors=[(vector_id, vector)], metadata=metadata)

#         # 4. Print vector ID and document name
#         print(f"Uploaded Document - Vector ID: {vector_id}, Document Name: {filename}")

# print("All documents uploaded successfully!")


In [36]:
keyword = """ summarize ai paper"""

In [37]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke(""" summarize ai paper""")
print(format_docs(retrieved_docs))

Abhishek, Amit Kumar Bindal and Dharminder Yadav

ARTICLES YOU MAY BE INTERESTED IN

Automated implementation of test scenarios by UML combinational diagrams via uniformed algorithm AIP Conference Proceedings 2576, 050012 (2022); https://doi.org/10.1063/5.0105807

A machine learning based assistive tool for finding potability of drinking water: An approach towards “Water for Life” AIP Conference Proceedings 2576, 050015 (2022); https://doi.org/10.1063/5.0105800

Asynchronous carry look ahead adder using energy efficient pipeline template AIP Conference Proceedings 2576, 050016 (2022); https://doi.org/10.1063/5.0107203

AIP Conference Proceedings 2576, 050014 (2022); https://doi.org/10.1063/5.0105723

© 2022 Author(s).

2576, 050014

Abhishek, Amit Kumar Bindal and Dharminder Yadav

ARTICLES YOU MAY BE INTERESTED IN

Automated implementation of test scenarios by UML combinational diagrams via uniformed algorithm AIP Conference Proceedings 2576, 050012 (2022); https://doi.org/10.1063/5.0

In [38]:
template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Provide concise and informative answers.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents 
5. Answer for normal conversation question like "Hi", "Hey", "Hello", "How are you", and many others questions with answer "Hello, How can I assist you?".
6. If question is on "summarize" or "summerization", then summarize the documents and (1/4)th the size of documents.

Question: {question}

Context: {context}

Answer:
"""
prompt = template.format(question = keyword, context =  format_docs(retrieved_docs))

In [39]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1.0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    #chain_type="stuff",
    retriever=vectorstore.as_retriever(),

)
qa.invoke(keyword)

{'query': ' summarize ai paper',
 'result': "The context provided does not include specific details about the content of the AI paper authored by Abhishek, Amit Kumar Bindal, and Dharminder Yadav. It only lists the paper's citation and related articles. To provide an accurate summary, I would need more information about the paper's objectives, methodology, findings, and conclusions. If you can provide more details or a specific section of the paper, I would be happy to help summarize it."}

In [40]:
custom_rag_template = PromptTemplate.from_template(template)

# Create the parallel chain
My_rag_chain = RunnableParallel(
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
) | custom_rag_template | llm | StrOutputParser()


## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()

In [41]:
keyword = """summarize ai paper"""
My_rag_chain.invoke(keyword)
#print("Chat with your Documents:")
print(My_rag_chain.invoke(keyword))

The answer is not specifically mentioned in the provided documents.


In [42]:
keyword = """summarize ai paper"""
My_rag_chain.invoke(keyword)
#print("Chat with your Documents:")
print(My_rag_chain.invoke(keyword))

Hello, How can I assist you?
